#Data 05: Personal Loan data
* Data from: [Personal Loan](https://www.kaggle.com/itsmesunil/bank-loan-modelling)  
Personal Loan 데이터는 고객을 더 늘리고자 하는 가상의 은행 데이터이다.  
이 회사의 경영진은 부채를 가진 고객을 개인 대출 고객으로 전환하는 방법을 모색하려고 한다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##1.데이터 둘러보기

In [3]:
#한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum # 글꼴 설치하기.
!sudo fc-cache -fv # 글꼴 캐시를 스캔한다.
!rm ~/.cache/matplotlib -rf # font 정보를 fontlist.json 에서 관리한다.
#런타임 다시 시작

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [16]:
import matplotlib.font_manager as fm # 폰트를 다루기 위한 모듈이다.
font_list=fm.findSystemFonts(fontpaths=None, fontext="ttf") # 지정된 경로에서 글꼴을 검색한다.
print(len(font_list))

27


In [22]:
font_list

['/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',
 '/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareR.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
 '/usr/shar

In [28]:
path=font_list[-1]
path

'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

In [34]:
font_name=fm.FontProperties(fname=path).get_name()
font_name

'NanumBarunGothic'

In [32]:
#기본 패키지 불러오기
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family=font_name) # 나눔 고딕 폰트 지정하기.
plt.rcParams['font.family'] = font_name 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [35]:
#구글 드라이브 마운트하기(이미 되어있다면 건너뛴다)
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
#데이터 불러오기
#데이터 파일 열어보면 sheet 1은 데이터 설명, sheet 2가 실제 데이터이므로 sheet 2만 가져온다!
bk = pd.read_excel('/content/Bank_Personal_Loan_Modelling.xlsx', sheet_name=1) # sheet_name=1

In [40]:
#불러온 데이터 일단 보기
bk.head(5)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


<컬럼 설명>

* ID: 고객 번호
* Age: 나이
* Experience: 사회 생활
* Income: 연간 수입 (달러 000)  
* Zipcode: 우편번호  
* Family: 가족 구성원 수
* CCAvg: 신용카드 월평균 이용 금액 (달러 000)  
* Education: 학력(1: Undergrad, 2: Graduate, 3: Advanced/Professional)
* Mortgage: 주택 담보 대출 금액(달러 000)
* Personal Loan: 지난번 영업에서 고객이 개인대출을 받았는가?(0: N, 1: Y)  
* Securities Account: 보험 유무(0: N, 1: Y)  
* CD Account: 양도성 예금증서 보유 유무(0: N, 1: Y)  
* Online: 온라인 뱅킹 유무(0: N, 1: Y)
* Credit Card: 신용카드 보유 유무(0: N, 1: Y)


* 범주형 변수: *빈도frequency 계산 가능*
               ID, zipcode, educaiont, personal loan, securities account, cd account, online, credit card
* 연속형 변수: *평균, 표준편차, min/max 등 수치 계산 가능*
               age, experience, income, family, ccavg, mortgage

In [44]:
bk["ID"].unique()

array([   1,    2,    3, ..., 4998, 4999, 5000])

In [42]:
#데이터 기본 정보 확인하기
bk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [45]:
#변수형 컬럼 안에는 어떤 내용이 들어있을까?
print('Education: ', bk['Education'].unique())
print('Personal Loan: ', bk['Personal Loan'].unique())
print('Securities Account: ', bk['Securities Account'].unique())
print('CD Account: ', bk['CD Account'].unique())
print('Online: ', bk['Online'].unique())
print('CreditCard: ', bk['CreditCard'].unique())

Education:  [1 2 3]
Personal Loan:  [0 1]
Securities Account:  [1 0]
CD Account:  [0 1]
Online:  [0 1]
CreditCard:  [0 1]


In [47]:
#연속형(수치) 데이터의 통계치 확인하기
bk.describe().transpose() # 보기가 더 편리하다.

,count,mean,std,min,25%,50%,75%,max
ID,5000.0,2500.500000,1443.520003,1.0,1250.75,2500.5,3750.25,5000.0
Age,5000.0,45.338400,11.463166,23.0,35.00,45.0,55.00,67.0
Experience,5000.0,20.104600,11.467954,-3.0,10.00,20.0,30.00,43.0
Income,5000.0,73.774200,46.033729,8.0,39.00,64.0,98.00,224.0
ZIP Code,5000.0,93152.503000,2121.852197,9307.0,91911.00,93437.0,94608.00,96651.0
Family,5000.0,2.396400,1.147663,1.0,1.00,2.0,3.00,4.0
CCAvg,5000.0,1.937913,1.747666,0.0,0.70,1.5,2.50,10.0
Education,5000.0,1.881000,0.839869,1.0,1.00,2.0,3.00,3.0
Mortgage,5000.0,56.498800,101.713802,0.0,0.00,0.0,101.00,635.0
Personal Loan,5000.0,0.096000,0.294621,0.0,0.00,0.0,0.00,1.0


In [48]:
#데이터 내용 정리해서 출력하기
print("총 데이터 수: ", bk.shape[0]*bk.shape[1])
print("총 결측치 수: {} = 전체 데이터의 {:.2f}% ".format(bk.isnull().sum().sum(), (bk.isnull().sum().sum()*100)/(bk.shape[0]*bk.shape[1])))
print("Thera bank 고객의 나이는 {}세부터 {}세까지 있으며 평균 나이는 {:.1f}세이다.".format((bk['Age'].min()),(bk['Age'].max()), (bk['Age'].mean())))
print("Thera bank 고객의 평균 수입은 {:.1f} $이며, 최고수입은 {} $로 평균의 {:.1f}배이다.".format((bk['Income'].mean()*1000), (bk['Income'].max()*1000), (bk['Income'].max()/(bk['Income'].mean()))))

총 데이터 수:  70000
총 결측치 수: 0 = 전체 데이터의 0.00% 
Thera bank 고객의 나이는 23세부터 67세까지 있으며 평균 나이는 45.3세이다.
Thera bank 고객의 평균 수입은 73774.2 $이며, 최고수입은 224000 $로 평균의 3.0배이다.


##2.질문하기

* 수입/월평균신용카드사용량/가족인원수와 대출 사이의 관계는?
* 월 평균 사용 금액을 구간화 해서 특징을 볼 수 있을까?
* 로지스틱 회귀 분석을 사용해서 신규 고객의 대출여부를 예측하는 모델 만들기 (목적)


##3.데이터 정비하기

###3-1. 컬럼명 바꾸기

In [ ]:
#중복 데이터 확인
bk.______________().sum()

In [ ]:
#컬럼명 확인하기
bk._________

In [ ]:
#컬럼명 변경하기
bk._________ = ['id', 'age', 'exp', 'income', 'zip', 'fam', 'ccavg',
       'edu', 'mort', 'loan', 'secu',
       'cd', 'online', 'cc']
bk._________

###3-2. 필요없는 컬럼 제거

In [ ]:
#id, zip code는 분석과 큰 연관관계가 없으므로 제거
bk1 = bk.____(['__','___'], axis=1, inplace=______)

In [ ]:
bk1.head()

##4.EDA & Visualization

###4-1. 간단한 분석

In [ ]:
#수입과 대출 사이의 관계 집계하기
df1 = bk1._________('____')['_____'].___([np.____,'____'])
df1.rename(columns={'mean': 'Income Mean','count':'Number of People'})

In [ ]:
#월평균 신용카드 사용액과 대출 사이의 관계 집계하기
df2 = bk1.______('____')['_____'].___([np.____,'_____'])
df2.rename(columns={'mean': 'CCAvg Mean','count':'Number of People'})

In [ ]:
#가족 인원수와 대출 사이의 관계 집계하기
df3 = bk1._____('___')['loan'].___([np.____,'_____'])
df3.rename(columns={'mean': 'Personal Loan Mean','count':'Number of People'})

###4-2. 변수의 분포 확인하기

In [ ]:
#연속형 변수의 분포 확인하기
f, ax = plt.subplots(2, 3, figsize=(17, 9))

sns._________(bk1['age'], ax=ax[_,_])
sns._________(bk1['exp'], ax=ax[_,_])
sns._________(bk1['income'], ax=ax[_,_])
sns._________(bk1['fam'], ax=ax[_,_])
sns._________(bk1['ccavg'], ax=ax[_,_])
sns._________(bk['mort'], ax=ax[_,_])
#sns._________(bk1['mort'][bk1['mort'] != 0], ax=ax[1,2]) #0인 것은 빼고 그리자!

plt.show()

In [ ]:
#Age, exp, income에 대해 좀 더 그래프를 자세히 그려보자
#세로선으로 평균값, 25, 50, 75% 선을 그어주자
def drawline(plt, col):
    mean = bk1.___________().loc['____', col]
    m25 = bk1.___________().loc['___', col]
    m50 = bk1.___________().loc['___', col]
    m75 = bk1.___________().loc['___', col]
    plt._______(____, color = 'red')
    plt._______(____, color = 'blue')
    plt._______(____, color = 'navy')
    plt._______(____, color = 'purple')
    plt.legend({'Mean': mean, '25%' : m25, '50%' : m50, '75%' : m75})

f, ax = plt.subplots(3, 1, figsize=(12, 13))

pp = sns._______(bk1['___'], ax = ax[0], bins=10, color = 'orange')
drawline(pp, 'age')
pp = sns._______(bk1['___'], ax = ax[1], bins=10, color = 'orange')
drawline(pp, 'exp')
pp = sns._______(bk1['______'], ax = ax[2], color = 'orange')
drawline(pp, 'income')

In [ ]:
#violin plot으로 범주형 변수 확인하기
f, ax = plt.________(1, 3, figsize=(16, 6))

sns._________(bk1['loan'],bk1['______'],ax=ax[0]);
sns._________(bk1['loan'],bk1['___'],ax=ax[1]);
sns._________(bk1['loan'],bk1['___'],ax=ax[2]);

plt.show()

#수입이 높을 때 personal loan을 받을 확률이 높다!

###4-3. 변수 간 상관관계 파악하기

In [ ]:
#Heat map을 그려서 변수 간 상관관계를 파악하자
f, ax = plt._________(1, 1, figsize=(15, 15))
sns.______(bk1____r(), ax=ax, annot=True, fmt='.2g')

#가장 상관관계가 높은 것은 나이와 경험(0.99).
#CCAvg와 수입은 중간 정도의 상관관계(0.65)를 가진다.
#수입은 CCAvg, Mortgage, CD Account, Personal Loan에 영향을 미친다

###4-4. 구간화 해서 그래프 그리기

In [ ]:
#신용카드 월평균 이용금액을 구간화 하자
df4 = bk1[['ccavg', 'cc', 'loan']]
df4['ccavg_bin'] = pd.___(df4['_____'], ____ = [_, _, _, _, ___], labels = ['0-2', '3-4', '5-6', '7+'])
df4

In [ ]:
#구간별로 신용카드 유무와 신용카드 월평균 사용액 집계
df5 = df4._______(['________', '__'])['_____'].___().___________()
df5

In [ ]:
#구간화 결과를 그래프로 나타내자(bar plot)
f, ax= plt._________(1, 2, figsize = (16, 6))

sns.______(df5['__________'], df5['_____'], hue = df5['__'], palette= "cividis", ax=ax[0]);
ax[0].set(xlabel = 'CC avg bins', ylabel = 'Count of customers');

#참조할 수 있도록 수입과 신용카드 월평균 사용액을 함께 나타내자
sns.____________(x='______', y='_____', data=bk1, ax=ax[1], hue = '____', palette= "cividis", alpha=0.6)

##5.로지스틱 회귀분석  
로지스틱 회귀분석(Logistic regression):  
종속변수와 독립변수 사이의 관계를 함수로 나타내어, 이 다음을 예측하는 방식.  
독립변수 사이의 선형 결합으로 종속변수를 설명하는 것은 선형 회귀와 동일하지만, 종속변수가 범주형 데이터를 대상으로 한다는 점이 다르다.

In [ ]:
#로지스틱 회귀분석에 필요한 sklearn 패키지 등등 불러오기
import os

from sklearn.linear_model import ________________
from sklearn._________________ import train_test_split
from sklearn import metrics
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import __________________, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import _____________

import statsmodels.api as sm
import itertools
import time

###5-1. 데이터 나누기
설명변수(X), 타겟변수(Y) 분리 및 학습데이터와 평가데이터 

* 회귀분석의 목표:  
어떤 특징을 가진 고객이 개인대출을 받을 것인가?

In [ ]:
#loan이 타겟변수, 나머지 컬럼이 설명변수가 된다
feature_columns = list(bk1.________.________________(['____']))
feature_columns

In [ ]:
#설명변수
X = bk1[_______________]
#타겟변수
y = bk1['____'] # 대출여부: 1 or 0

In [ ]:
#학습 데이터와 평가 데이터 나누기(7:3으로 나누자!)
#train: 학습 데이터, test: 평가 데이터
_______, _______, _______, _______ = ________________(_, _, stratify=y,train_size=___,test_size=___,random_state=42)
print(_______.shape, _______.shape, _______.shape, _______.shape)

In [ ]:
train_x

In [ ]:
#각 컬럼의 값들이 정규분포를 가지도록 scaling 해 주는 과정
scaler = ____________()
train_x = ______._____________(_______)
test_x = ______._____________(_______)

In [ ]:
train_x

###5-2. 모델 만들기

In [ ]:
#모델 만들기: 알고리즘 방식 = liblinear, iteration 최대 횟수 = 2000
lrm = _________________________(solver='_____________', ________=2000)

In [ ]:
#학습 데이터 집어넣기
lrm.___(_______, ________)

In [ ]:
#모델에서 산출된 계수 확인하기
coeff_df = pd.__________(lrm._______)
coeff_df['Intercept'] = lrm._________
coeff_df

###5-3. 모델링 결과 확인하기

In [ ]:
#모델에 학습 데이터를 넣어 정확도(accuracy) 확인하자
training_predict = lrm._______(_______)
print('학습 데이터를 넣었을 때 모델의 정확도: {:.4f}'.format(metrics._______________(_______, training_predict)))

In [ ]:
#학습 데이터로 만든 모델에 테스트 데이터를 넣어 어떤 분류가 되었는지 확인하자
test_predict = lrm._______(_______)
print('테스트 데이터를 넣었을 때 모델의 정확도: {:.4f}'.format(metrics._______________(_______, test_predict)))

###5-4. Confusion matrix & Report

In [ ]:
#confusion matrix 구하기
logistic_cm = metrics._____________________(_______, ________________, labels=[1,0])
print(logistic_cm)

In [ ]:
logistic_cm_df = pd._____________(_____________, index = [i for i in ['1','0']], columns = [i for i in ['predict 1', 'predict 0']])
logistic_cm_df

In [ ]:
f, ax= plt.subplots(1, 1, figsize = (10, 6))
plt.title('Confusion Matrix for Logistic Regression Model', size=15)
sns._______(_________________, annot=True, fmt='g',ax=ax)

#True Positives: 대출 제안을 받을 사람을 정확하게 예측  =  __
#True Negatives (TN): 대출 제안을 거절할 사람을 정확하게 예측  =  ____
#False Positives (FP): 대출 제안을 받을 것으로 잘못 판단(Type I error)  =  __
#False Negatives (FN): 대출 제안을 받지 않을 것으로 잘못 판단(Type II error)  =  __

In [ ]:
print('Logistic Regression Model - Classification Report')
print('')
print(metrics._____________________________(______, ______________, labels=[1,0]))

##6.Review

* Thera bank 데이터는:  
총 데이터 수:  70000  
총 결측치 수: 0 = 전체 데이터의 0.00%   
Thera bank 고객의 나이는 23세부터 67세까지 있으며 평균 나이는 45.3세이다.  
Thera bank 고객의 평균 수입은 73774.2 $이며, 최고수입은 224000 $로 평균의 3.0배이다.

* 수입/월평균신용카드사용량/가족인원수와 대출 사이의 관계는?  
   : 대출 받은 사람의 수입이 대출 받지 않은 사람보다 3배 정도 높다  
   : 대출받은 사람이 받지 않은 사람보다 월평균 신용카드 사용액이 2배 가량 높다  
   : 가족 인원수가 많을 수록 대출 금액이 올라가나 정비례 하지는 않는다  
* 월 평균 사용 금액을 구간화 해서 특징을 볼 수 있을까?
* 로지스틱 회귀 분석을 사용해서 신규 고객의 대출여부를 예측하는 모델 만들기
